<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential()  # identity인 경우
        if stride != 1:  # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes  # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset

data_transformations = {
    "basic": transforms.Compose([
        transforms.ToTensor()
    ]),
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["crop_flip"])
augmented_train_data = ConcatDataset((train_dataset, train_aug_dataset))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 75:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1640625
Current benign train loss: 2.304425001144409

Current batch: 100
Current benign train accuracy: 0.2890625
Current benign train loss: 1.8200820684432983

Current batch: 200
Current benign train accuracy: 0.359375
Current benign train loss: 1.6969637870788574

Current batch: 300
Current benign train accuracy: 0.4296875
Current benign train loss: 1.4296765327453613

Current batch: 400
Current benign train accuracy: 0.4765625
Current benign train loss: 1.447129249572754

Current batch: 500
Current benign train accuracy: 0.4453125
Current benign train loss: 1.479734182357788

Current batch: 600
Current benign train accuracy: 0.625
Current benign train loss: 1.2573543787002563

Current batch: 700
Current benign train accuracy: 0.46875
Current benign train loss: 1.4412521123886108

Total benign train accuarcy: 42.725
Total benign train loss: 1232.9048200845718

[ Test epoch: 0 ]

Test accuarcy: 45.58
Test average l


Test accuarcy: 83.95
Test average loss: 0.0048702706605196
Model Saved!

[ Train epoch: 8 ]

Current batch: 0
Current benign train accuracy: 0.9375
Current benign train loss: 0.22275249660015106

Current batch: 100
Current benign train accuracy: 0.890625
Current benign train loss: 0.2858642637729645

Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3402712345123291

Current batch: 300
Current benign train accuracy: 0.8671875
Current benign train loss: 0.39416268467903137

Current batch: 400
Current benign train accuracy: 0.8828125
Current benign train loss: 0.349543958902359

Current batch: 500
Current benign train accuracy: 0.8671875
Current benign train loss: 0.36895450949668884

Current batch: 600
Current benign train accuracy: 0.890625
Current benign train loss: 0.3146347403526306

Current batch: 700
Current benign train accuracy: 0.90625
Current benign train loss: 0.288381963968277

Total benign train accuarcy: 90.628
Total benign train los


Total benign train accuarcy: 93.279
Total benign train loss: 153.33111960440874

[ Test epoch: 15 ]

Test accuarcy: 85.78
Test average loss: 0.004584059269726277
Model Saved!

[ Train epoch: 16 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.17275160551071167

Current batch: 100
Current benign train accuracy: 0.921875
Current benign train loss: 0.19869834184646606

Current batch: 200
Current benign train accuracy: 0.984375
Current benign train loss: 0.0889689102768898

Current batch: 300
Current benign train accuracy: 0.890625
Current benign train loss: 0.2871628999710083

Current batch: 400
Current benign train accuracy: 0.953125
Current benign train loss: 0.18305331468582153

Current batch: 500
Current benign train accuracy: 0.921875
Current benign train loss: 0.22996950149536133

Current batch: 600
Current benign train accuracy: 0.9140625
Current benign train loss: 0.27328112721443176

Current batch: 700
Current benign train accuracy: 0.9140


Current batch: 700
Current benign train accuracy: 0.9609375
Current benign train loss: 0.14204643666744232

Total benign train accuarcy: 94.379
Total benign train loss: 129.86717293784022

[ Test epoch: 23 ]

Test accuarcy: 89.2
Test average loss: 0.0034280162096023557
Model Saved!

[ Train epoch: 24 ]

Current batch: 0
Current benign train accuracy: 0.9609375
Current benign train loss: 0.15264984965324402

Current batch: 100
Current benign train accuracy: 0.9609375
Current benign train loss: 0.1228276938199997

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.10329020768404007

Current batch: 300
Current benign train accuracy: 0.953125
Current benign train loss: 0.16046009957790375

Current batch: 400
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2670222520828247

Current batch: 500
Current benign train accuracy: 0.9609375
Current benign train loss: 0.17018245160579681

Current batch: 600
Current benign train accuracy: 0


Current batch: 600
Current benign train accuracy: 0.953125
Current benign train loss: 0.12032433599233627

Current batch: 700
Current benign train accuracy: 0.9765625
Current benign train loss: 0.09187142550945282

Total benign train accuarcy: 94.731
Total benign train loss: 120.49732723087072

[ Test epoch: 31 ]

Test accuarcy: 88.28
Test average loss: 0.004084680506587029
Model Saved!

[ Train epoch: 32 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.14343993365764618

Current batch: 100
Current benign train accuracy: 0.953125
Current benign train loss: 0.21211691200733185

Current batch: 200
Current benign train accuracy: 0.9375
Current benign train loss: 0.16708345711231232

Current batch: 300
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1648162603378296

Current batch: 400
Current benign train accuracy: 0.9453125
Current benign train loss: 0.12354664504528046

Current batch: 500
Current benign train accuracy: 0.976

...


Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.010721961036324501

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.003170367330312729

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0021313971374183893

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0029809968546032906

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0030264481902122498

Total benign train accuarcy: 99.814
Total benign train loss: 5.643193019961473

[ Test epoch: 63 ]

Test accuarcy: 94.54
Test average loss: 0.0021589514148421586
Model Saved!

[ Train epoch: 64 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.010862491093575954

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.010306648910045624

Current batch: 200
Current benign train accuracy: 0.9921875
Current 


Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.010686799883842468

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.002254708670079708

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0017470733728259802

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.001168744987808168

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.001737273414619267

Total benign train accuarcy: 99.878
Total benign train loss: 3.902216350805247

[ Test epoch: 71 ]

Test accuarcy: 94.59
Test average loss: 0.0021207697405479847
Model Saved!

[ Train epoch: 72 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0011637575225904584

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.004289041738957167

Current batch: 200
Current benign train accuracy: 1.0
Current benign train 


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0010715735843405128

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0061756013892591

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.01463728304952383

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.007637450471520424

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0009865263709798455

Total benign train accuarcy: 99.965
Total benign train loss: 1.9588839658827055

[ Test epoch: 79 ]

Test accuarcy: 94.93
Test average loss: 0.0020621923586819323
Model Saved!

[ Train epoch: 80 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.006210093852132559

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0008228590013459325

Current batch: 200
Current benign train accuracy: 1.0
Current benign train l


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0007837424054741859

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.002496479544788599

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.00412781210616231

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.00533759081736207

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0012767503503710032

Total benign train accuarcy: 99.965
Total benign train loss: 1.7905967480910476

[ Test epoch: 87 ]

Test accuarcy: 95.02
Test average loss: 0.0020377682075835763
Model Saved!

[ Train epoch: 88 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0015074509428814054

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0037417893763631582

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss:


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0007020822959020734

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.002616132842376828

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0007710091304033995

Current batch: 600
Current benign train accuracy: 0.9921875
Current benign train loss: 0.010479457676410675

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0015718796057626605

Total benign train accuarcy: 99.971
Total benign train loss: 1.5352233783632983

[ Test epoch: 95 ]

Test accuarcy: 94.95
Test average loss: 0.002008887079777196
Model Saved!

[ Train epoch: 96 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0009944071061909199

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.000564842950552702

Current batch: 200
Current benign train accuracy: 1.0
Current benign trai